# Goal Oriented Generation 

Based on prompts from [Creating Suspenseful Stories with Large Language Models](https://arxiv.org/abs/2402.17119).

In [30]:
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

# See langchain_aws for more information on other bedrock models
# This is the model we will be using for this example

model = ChatBedrock(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    model_kwargs={"temperature": 0.7},
)

# Setting a role for the chat model to act as a short story writer

role = """You are a short story writer with a passion for creative writing and storytelling. 
Your task is to create engaging stories that captivate your audience and leave them wanting more. 
"""

def prompt_from_template(template: str):
    return ChatPromptTemplate.from_messages([
        ("system", role),
        ("human", template)
    ])


# Background Setup

This is the world knowledge needed to tell a coherent story about the protagonist.  We'll need the following items:

- Protagonist's name and occupution
- The goal they want to achieve
- The consequence for not achieving the goal

From those items, we'll prompt the model to write an introductory paragraph


In [31]:
genre = "Espionage Thriller"
name_pormpt = prompt_from_template(template="Make up the name and the occupation of a protagonist of a {genre}.")

name_chain = name_pormpt | model | StrOutputParser()
name_occupation = name_chain.invoke({"genre":genre})
print(name_occupation)

Here is the name and occupation of the protagonist for an espionage thriller:

Name: Evelyn Sinclair
Occupation: CIA Intelligence Analyst

Evelyn Sinclair is a brilliant and highly skilled CIA intelligence analyst who specializes in deciphering complex data and connecting the dots to uncover hidden threats. With a razor-sharp mind and an unparalleled attention to detail, she is the agency's go-to expert when it comes to untangling the most intricate espionage cases.

Despite her desk-bound role, Evelyn is no stranger to the high-stakes world of covert operations. Her keen analytical abilities and ability to think several steps ahead of her adversaries have made her an invaluable asset to the CIA, often called upon to provide crucial intelligence that informs the decisions of field agents and senior leadership.

When a new case emerges that threatens to unravel the delicate balance of global power, Evelyn finds herself thrust into the center of a dangerous web of lies, betrayal, and int

In [32]:


goal_prompt = prompt_from_template(template="<name_occupation>{name_occupation}</name_occupation>\n\nGiven the <name_occupation>, create a very concrete goal the protagonist wants to achieve")

gaol_chain = goal_prompt | model | StrOutputParser()
goal = gaol_chain.invoke({"name_occupation":name_occupation})
print(goal)

Given the background provided for the protagonist, Evelyn Sinclair, a CIA intelligence analyst, her concrete goal in this espionage thriller could be:

To uncover the truth behind a complex international conspiracy and prevent a devastating global catastrophe.

As a highly skilled and experienced intelligence analyst, Evelyn's goal is to use her exceptional analytical abilities to meticulously investigate the emerging case, connect the dots, and ultimately identify the key players and their nefarious plans. With the fate of the world hanging in the balance, Evelyn is driven to overcome any obstacles and unravel the intricate web of deception to stop the impending disaster.

Her unwavering determination to protect her country and the global community at large fuels her pursuit of the truth, even as she finds herself thrust into the perilous world of covert operations, where the line between ally and enemy becomes increasingly blurred. Evelyn's goal is not just to gather the necessary in

In [33]:

situtaion = """<story_background>
<name_occupation>{name_occupation}</cname_occupation> 
<goal>{goal}</goal>
</story_background>
Tell me about a dire situation that the protagonist would be put in, if the protagonist couldn’t achieve the goal."""

situtaion_prompt = prompt_from_template(template=situtaion)

situation_chain = situtaion_prompt | model | StrOutputParser()
dire_situation = situation_chain.invoke({"name_occupation":name_occupation, "goal":goal})

print(dire_situation)

If Evelyn Sinclair, the CIA intelligence analyst, fails to uncover the truth behind the complex international conspiracy and prevent the impending global catastrophe, the consequences could be truly dire:

Escalating Global Tensions: Without Evelyn's crucial intelligence and interventions, the web of lies and deception would continue to unravel, leading to heightened tensions between the world's superpowers. This could result in a dangerous standoff, with the threat of military conflict looming large.

Devastating Attacks: If the enemy's plans are not thwarted, they could proceed with their nefarious schemes, potentially orchestrating a series of devastating attacks on strategic locations or critical infrastructure around the world. This could lead to massive loss of life, widespread destruction, and a global state of panic and chaos.

Collapse of the International Order: The successful execution of the conspiracy could undermine the fragile balance of global power, eroding trust betwe

In [34]:
intro_instruction =  """<story_background>
<name_occupation>{name_occupation}</cname_occupation> 
<goal>{goal}</goal>
<situation>{dire_situation}</situation>
</story_background>
Write an introductory paragraph of a story that covers all the above information.
"""

intro_prompt = prompt_from_template(template=intro_instruction)
intro_chain = intro_prompt | model | StrOutputParser()
intro = intro_chain.invoke({"name_occupation":name_occupation, "goal":goal, "dire_situation":dire_situation})
print(intro)

Here is an introductory paragraph for the espionage thriller story featuring CIA intelligence analyst Evelyn Sinclair:

Evelyn Sinclair's razor-sharp mind was her greatest asset, honed by years of painstaking analysis and an unwavering commitment to unraveling the most complex espionage cases. As the CIA's top intelligence analyst, she was the agency's go-to expert, her keen eye for detail and ability to connect the dots proving invaluable in a world where the line between ally and enemy was constantly blurred. But as Evelyn found herself thrust into the center of a dangerous web of international intrigue, she knew the stakes had never been higher. A new case had emerged, one that threatened to unravel the delicate balance of global power, and with time running out, Evelyn would have to use every tool in her arsenal to track down the elusive enemy and prevent a devastating outcome that could have catastrophic consequences for the entire world.
